In [2]:
import tensorflow as tf


In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
#initialising weights randomly

def init_weights(shape):
    init_random_distance = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_distance)

In [7]:
#initialising bias randomly

def init_bias(shape):
    init_bias_value = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_value)

In [8]:
#setting stride and padding

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [9]:
#defining max pooling layer

def max_pooling(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [10]:
#convolution layer with relu as activation function

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [11]:
#fully connected layer

def full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [12]:
#input image of size 28*28*1

x = tf.placeholder(tf.float32,shape=[None,784])

In [13]:
#actual output

y_true = tf.placeholder(tf.float32,shape=[None,10])

In [14]:
x_image = tf.reshape(x,[-1,28,28,1])

In [15]:
convolution_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convolution_1_pooling = max_pooling(convolution_1)

In [16]:
convolution_2 = convolutional_layer(convolution_1_pooling,shape=[6,6,32,64])
convolution_2_pooling = max_pooling(convolution_2)

In [17]:
convolution_2_flat = tf.reshape(convolution_2_pooling,[-1,7*7*64])
full_layer_1 = tf.nn.relu(full_layer(convolution_2_flat,1024))

In [18]:
hold_probability = tf.placeholder(tf.float32)
full_dropout = tf.nn.dropout(full_layer_1,keep_prob=hold_probability)

In [19]:
y_pred = full_layer(full_dropout,10)

In [22]:
#defining loss function

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [23]:
#optimisation

optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
training = optimizer.minimize(cross_entropy)

In [24]:
#initialising

init = tf.global_variables_initializer()

In [ ]:
#session

step_size = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(step_size):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(training,feed_dict={x:batch_x,y_true:batch_y,hold_probability:0.5})
        
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            check_match = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(check_match,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_probability:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
